# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.13.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set
* apply few transformations to imgaes like rescaling, zoom-in, zoom-out, h-flip etc... so that model can not overfit with these images

* This is called as Image Augmentation

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,    ### rescale: feature scaling in image analysis
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),  ### image size when imported
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

* we are not applying transfomation to test images bcz these are new data points(images) to test the mode performance

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)  ### Normalize test images


test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,    ### test images in batches as train images are in batches
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [25]:
#test_set[0]

(array([[[[0.03529412, 0.06666667, 0.02352941],
          [0.03529412, 0.04705883, 0.01176471],
          [0.21176472, 0.25882354, 0.20392159],
          ...,
          [0.12941177, 0.17254902, 0.15686275],
          [0.13725491, 0.16078432, 0.15294118],
          [0.11764707, 0.14117648, 0.13333334]],
 
         [[0.2627451 , 0.29411766, 0.2509804 ],
          [0.02745098, 0.0509804 , 0.01176471],
          [0.03137255, 0.0627451 , 0.01176471],
          ...,
          [0.13725491, 0.18039216, 0.16470589],
          [0.13333334, 0.1764706 , 0.16078432],
          [0.1137255 , 0.15686275, 0.14117648]],
 
         [[0.00784314, 0.03137255, 0.        ],
          [0.25490198, 0.29411766, 0.2509804 ],
          [0.0627451 , 0.07058824, 0.02745098],
          ...,
          [0.13333334, 0.1764706 , 0.16078432],
          [0.12156864, 0.18039216, 0.16078432],
          [0.11764707, 0.1764706 , 0.15686275]],
 
         ...,
 
         [[0.2784314 , 0.30980393, 0.227451  ],
          [0.36078

## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
        
## 2D convolution layer : This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs.
# filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
#  kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 5)

Epoch 1/5
250/250 [==============================] - 44s 177ms/step - loss: 0.5551 - accuracy: 0.7155 - val_loss: 0.5300 - val_accuracy: 0.7455
Epoch 2/5
250/250 [==============================] - 46s 185ms/step - loss: 0.5165 - accuracy: 0.7396 - val_loss: 0.5217 - val_accuracy: 0.7470
Epoch 3/5
250/250 [==============================] - 45s 180ms/step - loss: 0.5080 - accuracy: 0.7508 - val_loss: 0.4840 - val_accuracy: 0.7625
Epoch 4/5
250/250 [==============================] - 43s 171ms/step - loss: 0.4892 - accuracy: 0.7595 - val_loss: 0.5019 - val_accuracy: 0.7615
Epoch 5/5
250/250 [==============================] - 49s 198ms/step - loss: 0.4663 - accuracy: 0.7763 - val_loss: 0.4721 - val_accuracy: 0.7735


## Making a single prediction

In [17]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)  ### get extra dimesion since training done over batches as extra dimension
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 21ms/step


In [18]:
print(prediction)

cat


In [26]:
#training_set.class_indices

{'cats': 0, 'dogs': 1}